In [1]:
# model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

class ConvAutoencoder:
  @staticmethod
  def build(width, height, depth, filters=(32, 64), latent_dim=16):

    # initialize input shape to be channels last along with channels dimension itself
    input_shape = (height, width, depth)
    chan_dim = -1

    # define input to encoder
    inputs = Input(shape=input_shape)
    x = inputs

    # loop over the number of filters
    for f in filters:
      # apply CONV => RELU => BN operations
      x = Conv2D(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

    # flatten the network and then consturct our latent vector
    volume_size = K.int_shape(x)
    x = Flatten()(x)
    latent = Dense(latent_dim)(x)

    # build the encoder model
    encoder = Model(inputs, latent, name='encoder')

    # start building the decoder model which will accept output of encoder as its input
    latent_input = Input(shape=(latent_dim,))
    x = Dense(np.prod(volume_size[1:]))(latent_input)
    x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

    # loop over number of filters again but this time in reverse order
    for f in filters[::-1]:
      
      # apply a Conv_transpose => RELU => BN operation
      x = Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

    # apply a single Conv_Transpose layer used to recover the original depth of image
    x = Conv2DTranspose(depth, (3, 3), padding='same')(x)
    outputs = Activation('sigmoid')(x)

    # build decoder model
    decoder = Model(latent_input, outputs, name='decoder')

    # autoencoder is encoder + decoder
    autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')

    return (encoder, decoder, autoencoder)

In [2]:
# training autoencoders

import matplotlib
matplotlib.use('Agg')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

# Function to help visualize predictions made by unsupervised autoencoder 
def visualize_prediction(decode, gt, samples=10):
  # initialize list of output images
  outputs = None

  # loop over our number of output samples
  for i in range(0, samples):
    # grab original image and reconstructed image
    original = (gt[i]*255).astype('uint8')
    recon = (decode[i]*255).astype('uint8')

    # stack original and reconstructed images side by side
    output = np.hstack([original, recon])

    # if the output array is empty, initialize it as the current side-by-side image display
    if outputs is None:
      outputs = output

    # otherwise vertically stack the output
    else:
      outputs = np.vstack([outputs, output])

  return outputs

# arguments
model_name = 'output/autoencoder.model'         # path to our output trained autoencoder
visu = "recon_vis.png"
plot = "plot.png"
epochs = 20
init_lr = 1e-3
batch_size = 32

# load mnist dataset
print('[INFO] loading MNIST dataset...')
((trainX, _), (testX, _)) = mnist.load_data()

# add channel dimension to every image in dataset and scale pixel intensities to range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

# construct our Conv autoencoder
print('[INFO] building autoencoder...')
(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1)
opt = Adam(learning_rate=init_lr, decay=init_lr/epochs)
autoencoder.compile(loss='mse', optimizer=opt)

# train convolution autoencoder
H=autoencoder.fit(trainX, trainX, validation_data=(testX, testX), epochs=epochs, 
                    batch_size=batch_size)

# use the convolution auto encoder to make the predictions on the testing images, construct the 
# visualizations and save it to disk
decoded = autoencoder.predict(testX)
vis = visualize_prediction(decoded, testX)
cv.imwrite(visu, vis)

# construct a plot that plots and saves training history
N = np.arange(0, epochs)
plt.style.use('ggplot')
plt.figure()
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='validation_loss')
plt.title('Training loss and accuracy')
plt.xlabel('Epochs #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='lower left')
plt.savefig(plot)

# serialize the autoencoder model to disk
print('[INFO] saving autoencoder')
autoencoder.save(model_name, save_format='h5')

[INFO] loading MNIST dataset...
[INFO] building autoencoder...
Epoch 1/20
1875/1875 [==============================] - 18s 6ms/step - loss: 0.0188 - val_loss: 0.0115
Epoch 2/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0103 - val_loss: 0.0093
Epoch 3/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0092 - val_loss: 0.0089
Epoch 4/20
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0086 - val_loss: 0.0084
Epoch 5/20
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0082 - val_loss: 0.0081
Epoch 6/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0079 - val_loss: 0.0077
Epoch 7/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 8/20
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 9/20
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0073 - val_loss: 0.0074
Epoch 10/2